# Feature Selection using LASSO 

# Fire up Turi Create

In [1]:
import turicreate

/home/wsl-fanboy/turicreate/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.331326     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.448960     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
coef = model_all.coefficients
coef

name,index,value,stderr
(intercept),None,274873.05595049576,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.531086910105,None
sqft_living,None,24.42072098244546,None
sqft_living_sqrt,None,350.06055338605546,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


In [7]:
non_zero_weight = coef[coef["value"] > 0]
non_zero_weight

name,index,value,stderr
(intercept),None,274873.05595049576,None
bathrooms,None,8468.531086910105,None
sqft_living,None,24.42072098244546,None
sqft_living_sqrt,None,350.06055338605546,None
grade,None,842.0680348976231,None
sqft_above,None,20.024722417091112,None


In [8]:
non_zero_weight.print_rows(20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|      grade       |  None | 842.0680348976231  |  None  |
|    sqft_above    |  None | 20.024722417091112 |  None  |
+------------------+-------+--------------------+--------+
[6 rows x 4 columns]



# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [10]:
import numpy as np

validation_rss = {}
for l1_penalty in np.logspace(1, 7, num=13):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss

print(min(validation_rss.items(), key=lambda x: x[1]))

(10.0, 625766285142461.2)


In [11]:
model_test = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=10.0)
predictions_test = model.predict(testing)
residuals_test = testing['price'] - predictions_test
rss_test = sum(residuals_test**2)
print(rss_test)

156972779668688.66


In [12]:
non_zero_weight_test = model_test.coefficients[model_test.coefficients["value"] > 0]
print(model_test.coefficients["value"].nnz())
print()
non_zero_weight_test.print_rows(num_rows=20)

18

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)

In [15]:
coef_dict = {}
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target ='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    coef_dict[l1_penalty] = model.coefficients['value'].nnz()

In [16]:
import pprint
pprint.pprint(coef_dict)

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [17]:
l1_penalty_min = 2976351441.6313128
l1_penalty_max = 3792690190.7322536

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [18]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [19]:
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss, model.coefficients['value'].nnz()

In [20]:
pprint.pprint(validation_rss)

{2976351441.631313: (966925692362086.5, 10),
 3019316638.952415: (974019450084556.8, 10),
 3062281836.273517: (981188367942450.6, 10),
 3105247033.5946193: (989328342459471.6, 10),
 3148212230.9157214: (998783211265885.6, 10),
 3191177428.2368236: (1008477167020096.1, 10),
 3234142625.5579257: (1018298780553818.1, 10),
 3277107822.879028: (1028247992205975.9, 10),
 3320073020.20013: (1034616909232828.9, 8),
 3363038217.521232: (1038554735941040.0, 8),
 3406003414.8423343: (1043237237871703.9, 8),
 3448968612.1634364: (1046937488751713.5, 7),
 3491933809.4845386: (1051147625612863.0, 7),
 3534899006.8056407: (1055992735342998.9, 7),
 3577864204.126743: (1060799531763289.9, 7),
 3620829401.447845: (1065707689498229.5, 6),
 3663794598.768947: (1069464335425583.8, 6),
 3706759796.0900493: (1073504549585594.4, 6),
 3749724993.4111514: (1077632775581411.9, 6),
 3792690190.7322536: (1081867592324111.6, 6)}


In [21]:
bestRSS = float('inf')
for k,v in validation_rss.items():    
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
        bestl1 = k
        
print(bestRSS, bestl1)

1046937488751713.5 3448968612.1634364


In [22]:
model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3448968612.16)

In [23]:
non_zero_weight_test = model.coefficients[model.coefficients["value"] > 0]
non_zero_weight_test.print_rows(num_rows=8)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.1925443279  |  None  |
|     bedrooms     |  None | 661.7227177822278  |  None  |
|    bathrooms     |  None |  15873.957259268   |  None  |
|   sqft_living    |  None | 32.41022145125974  |  None  |
| sqft_living_sqrt |  None | 690.1147733133166  |  None  |
|      grade       |  None | 2899.420269749878  |  None  |
|    sqft_above    |  None | 30.011575302201138 |  None  |
+------------------+-------+--------------------+--------+
[7 rows x 4 columns]

